In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
df = pd.read_csv("C:/Users/matthewthomas/US_AQI.csv")

In [3]:
df.head()

,Unnamed: 0,CBSA Code,Date,AQI,Category,Defining Parameter,Number of Sites Reporting,city_ascii,state_id,state_name,lat,lng,population,density,timezone
0,0,10140,2022-01-01,21,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
1,1,10140,2022-01-02,12,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
2,2,10140,2022-01-03,18,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
3,3,10140,2022-01-04,19,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles
4,4,10140,2022-01-05,17,Good,PM2.5,2,Aberdeen,WA,Washington,46.9757,-123.8094,16571.0,588.0,America/Los_Angeles


In [4]:
df.columns

Index(['Unnamed: 0', 'CBSA Code', 'Date', 'AQI', 'Category',
       'Defining Parameter', 'Number of Sites Reporting', 'city_ascii',
       'state_id', 'state_name', 'lat', 'lng', 'population', 'density',
       'timezone'],
      dtype='object')

In [8]:
df.drop(["Unnamed: 0", "CBSA Code", "Category", "Number of Sites Reporting", "state_id", "timezone"], axis = 1, inplace = True)

In [9]:
df.columns

Index(['Date', 'AQI', 'Defining Parameter', 'city_ascii', 'state_name', 'lat',
       'lng', 'population', 'density'],
      dtype='object')

In [14]:
df = df[['Date', 'Defining Parameter', 'city_ascii', 'state_name', 'lat',
       'lng', 'population', 'density', "AQI"]]
X = df.iloc[:, 0:8].values
y = df.iloc[:, 8].values
print(pd.DataFrame(X[:10]))
print(pd.DataFrame(y[:10]))

            0      1         2           3        4         5        6      7
0  2022-01-01  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
1  2022-01-02  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
2  2022-01-03  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
3  2022-01-04  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
4  2022-01-05  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
5  2022-01-06  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
6  2022-01-07  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
7  2022-01-08  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
8  2022-01-09  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
9  2022-01-10  PM2.5  Aberdeen  Washington  46.9757 -123.8094  16571.0  588.0
    0
0  21
1  12
2  18
3  19
4  17
5  15
6  20
7  13
8  16
9  18


In [19]:
df['Date'] = pd.to_numeric(df['Date'])

In [20]:
df.dtypes

Date                    int64
Defining Parameter     object
city_ascii             object
state_name             object
lat                   float64
lng                   float64
population            float64
density               float64
AQI                     int64
dtype: object

In [10]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [29]:
ct = ColumnTransformer([("encoder", OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X = ct.fit_transform(X)
print(pd.DataFrame(X[:10]))

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.